In [1]:
!pip install uproot

In [2]:
# !pip install -q torch-scatter
# !pip install -q torch-sparse
# !pip install -q torch-geometric

In [3]:
# !pip install uproot tqdm numexpr google-colab

In [4]:
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import tqdm
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from keras import initializers
from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras
from matplotlib import pyplot as plt
from IPython.display import clear_output
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import pickle
import torch
from torch_geometric.data import Data
# import torch_geometric
# import tqdm
# import torch.nn as nn
# import torch_geometric.nn as pyg_nn
# from torch_geometric.loader import DataLoader
# import torch.optim as optim
# import torch.nn.functional as F

In [5]:
particles_file = uproot.open("/home/jonas/Documents/physics/lhcb/dfei/irishep/data/IRIS-HEP_DFEI/particles_data.root")

In [6]:
particles_tree = particles_file['Particles']

In [ ]:
relevant_particles_keys = ['ExpandedEventNumber',
                          'ExpandedParticleIndex',
                          'ExpandedPrimaryHeavyHadronIndex',
                          'FromPrimaryHeavyHadron',
                          'ParticleType',
                          'associated_ip_reco',
                          'associated_xPV_reco',
                          'associated_yPV_reco',
                          'associated_zPV_reco',
                          'charge',
                          'inGeomAcc',
                          'isCharged',
                          'pt_reco',
                          'px_reco',
                          'py_reco',
                          'pz_reco',
                          'xProd_reco',
                          'yProd_reco',
                          'zProd_reco']

In [ ]:
df_particles = particles_tree.arrays(relevant_particles_keys, library='pd')

In [ ]:
df_particles

In [ ]:
df_Y = df_particles.loc[:, ['ExpandedEventNumber', 'ExpandedParticleIndex', 'FromPrimaryHeavyHadron']]

In [ ]:
df_Y

In [ ]:
df_particles = df_particles.set_index('ExpandedParticleIndex')

In [ ]:
df_particles

In [ ]:
num_of_events = 6161

In [ ]:
relevant_edge_features = [
                          'DOCA_reco',
                          'FromSameAssociatedPV_reco',
                          'delta_z0_reco',
                          'p1_isCharged',
                          'p2_isCharged',
                          'theta_reco',
                          'trdist_reco'
                          ]

In [ ]:
K = 20

In [ ]:
list_of_graphs = []

In [ ]:
for event_number in tqdm.tqdm(range(num_of_events)):
  
  df_p = df_particles.loc[df_particles.ExpandedEventNumber == event_number]
  mapping = {index: i for i, index in enumerate(df_p.index)}
  x = df_p.loc[:, ~df_p.columns.isin(['ExpandedParticleIndex', 'ExpandedEventNumber', 'FromPrimaryHeavyHadron', 'ExpandedPrimaryHeavyHadronIndex', 'ParticleType', 'inGeomAcc'])]
  x = torch.tensor(x.values, dtype=torch.float)

  filename = f"event_{event_number}_top_{K}_relations_filtered_by_DOCA_for_each_particle.csv"
  path = f'/home/jonas/Documents/physics/lhcb/dfei/irishep/data/IRIS-HEP_DFEI/filtered_relations_by_DOCA/{filename}'
  df_r = pd.read_csv(path)
  edges = []
  edge_attr = []
  for _, row in tqdm.tqdm(df_r.iterrows()):
    sender = mapping[row['FirstParticleIndex']]
    reciever = mapping[row['SecondParticleIndex']]
    edge_1 = [sender, reciever]
    edge_2 = [reciever, sender]
    edges.append(edge_1)
    edges.append(edge_2)
    edge_features = list(row[relevant_edge_features].values)
    edge_attr.append(edge_features)
    edge_attr.append(edge_features)
  edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
  edge_attr = torch.tensor(edge_attr, dtype=torch.float)


  y = torch.tensor(df_Y.loc[df_Y.ExpandedEventNumber == event_number]['FromPrimaryHeavyHadron'].values)
  y = y.reshape(-1, 1)

  graph = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

  list_of_graphs.append(graph)

In [ ]:
import pickle 

with open(f'/home/jonas/Documents/physics/lhcb/dfei/irishep/data/IRIS-HEP_DFEI/list_of_graphs_from_filtered_data_k{K}.pickle', 'wb') as handle:
    pickle.dump(list_of_graphs, handle, protocol=pickle.HIGHEST_PROTOCOL)